In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import transformers

print(transformers.__version__)

4.35.2


In [ ]:
!pip install accelerate -U
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install sentencepiece

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
model_checkpoint = "VietAI/vit5-base-vietnews-summarization"

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
from datasets import load_dataset
from evaluate import load

raw_datasets = load_dataset("toanduc/paper-gpt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

In [ ]:
from transformers import BertTokenizer,BertModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = "summarize "

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Original Text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["Summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/1074 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

In [ ]:
# batch_size = 4
model_name = model_checkpoint.split("/")[-1]
fine_tuned_model_name = f"{model_name}-finetuned"

# args = Seq2SeqTrainingArguments(
#     fine_tuned_model_name,
#     evaluation_strategy="no",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     weight_decay=0.01,
#     save_total_limit=3,
#     num_train_epochs=6,
#     predict_with_generate=True,
#     fp16=True,
# )

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# trainer = Seq2SeqTrainer(
#     model,
#     args,
#     train_dataset=tokenized_datasets["train"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

In [ ]:
# trainer.train()

In [ ]:
num_epochs = 30
batch_size = 4

for epoch in range(num_epochs):
    print(f"Training Epoch {epoch + 1}/{num_epochs}")

    # Thiết lập trainer cho mỗi epoch
    args = Seq2SeqTrainingArguments(
        fine_tuned_model_name,
        evaluation_strategy="no",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=1,  # Chỉ chạy một epoch mỗi lần
        predict_with_generate=True,
        fp16=True,
    )
    trainer = Seq2SeqTrainer(
        model,
        args,
        train_dataset=tokenized_datasets["train"],
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Huấn luyện mô hình trong một epoch
    trainer.train()

    # Lưu trạng thái của mô hình sau mỗi epoch
    model.save_pretrained(fine_tuned_model_name)
    tokenizer.save_pretrained(fine_tuned_model_name)

# Kết thúc
print("Training finished.")

Training Epoch 1/30


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


Training Epoch 2/30


Step,Training Loss


...

Training Epoch 29/30


Step,Training Loss


Training Epoch 30/30


Step,Training Loss


Training finished.


In [ ]:
trainer.push_to_hub()

events.out.tfevents.1705284070.97c12fea2612.627.0:   0%|          | 0.00/4.94k [00:00<?, ?B/s]

events.out.tfevents.1705284217.97c12fea2612.627.1:   0%|          | 0.00/4.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/904M [00:00<?, ?B/s]

Upload 33 LFS files:   0%|          | 0/33 [00:00<?, ?it/s]

events.out.tfevents.1705284362.97c12fea2612.627.2:   0%|          | 0.00/4.94k [00:00<?, ?B/s]

...

events.out.tfevents.1705288150.97c12fea2612.627.28:   0%|          | 0.00/4.94k [00:00<?, ?B/s]

events.out.tfevents.1705288295.97c12fea2612.627.29:   0%|          | 0.00/4.94k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/toanduc/vit5-base-vietnews-summarization-finetuned/commit/cd667c0d9c50653861387c552267a6f3d15af34a', commit_message='End of training', commit_description='', oid='cd667c0d9c50653861387c552267a6f3d15af34a', pr_url=None, pr_revision=None, pr_num=None)